In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import re
import pickle
from typing import List, Dict, Tuple
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

In [3]:
try:
    from connector import get_db_connection
    print("Database connector imported successfully!")
except ImportError as e:
    print(f"Error importing database connector: {e}")
    print("Make sure your connector.py is in the path")

Database connector imported successfully!


In [4]:
#fetch unique products
def fetch_all_products():
    """Fetch all unique products from your database."""
    try:
        engine = get_db_connection()
        
        query = """
        SELECT DISTINCT productName 
        FROM sales_data 
        WHERE productName IS NOT NULL 
        AND productName != ''
        ORDER BY productName
        """
        
        df = pd.read_sql(query, engine)
        engine.dispose()
        
        products = df['productName'].tolist()
        print(f"Fetched {len(products)} unique products from database")
        return products
        
    except Exception as e:
        print(f"Error fetching products: {e}")
        return []

In [5]:
def fetch_sample_sales_data():
    """Fetch sample sales data for testing consolidation."""
    try:
        engine = get_db_connection()
        
        # Get latest date
        last_date_query = "SELECT MAX(orderDate) FROM product_sales;"
        last_date = pd.read_sql(last_date_query, engine).iloc[0, 0]
        
        # Get top 50 products by sales for the latest date
        query = """
        SELECT 
            productName AS `Product Name`,
            totalProductPrice,
            quantity AS `Quantity Sold`
        FROM sales_data 
        WHERE orderDate = %s
        AND productName IS NOT NULL
        ORDER BY totalProductPrice DESC
        LIMIT 50
        """
        
        df = pd.read_sql(query, engine, params=(last_date,))
        engine.dispose()
        
        # Add serial number
        df.insert(0, 'S.No', range(1, len(df) + 1))
        
        print(f"Fetched sample sales data: {len(df)} products")
        return df
        
    except Exception as e:
        print(f"Error fetching sales data: {e}")
        return pd.DataFrame()


In [6]:
print("Loading product data from database...")
products = fetch_all_products()
sample_sales = fetch_sample_sales_data()

Loading product data from database...
Fetched 47490 unique products from database
Fetched sample sales data: 50 products


#### Excel Export Refined Data

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from rapidfuzz import fuzz
from tqdm import tqdm
import math

MYSQL_USER = "root"
MYSQL_PASS = "root"
MYSQL_HOST = "localhost"
MYSQL_PORT = 3306
MYSQL_DB   = "sales_data"

engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASS}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
)

query = """
SELECT productName, totalProductPrice, quantity, brandName, categoryName
FROM sales_data
WHERE orderDate BETWEEN '2025-07-01' AND '2025-07-31'
AND productName IS NOT NULL
"""

df = pd.read_sql(query, engine)
df['totalProductPrice'] = pd.to_numeric(df['totalProductPrice'], errors='coerce')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df.dropna(subset=['totalProductPrice', 'quantity'], inplace=True)

def normalize_name(name):
    return (
        str(name).lower()
        .replace("™", "")
        .replace("®", "")
        .replace("-", " ")
        .replace("&", "and")
        .strip()
    )

df['cleanName'] = df['productName'].apply(normalize_name)
unique_clean_names = df['cleanName'].unique().tolist()

name_map = df[['cleanName', 'productName']].drop_duplicates().set_index('cleanName')['productName'].to_dict()

clusters = []
visited = set()

print("Clustering similar product names...")

for name in tqdm(unique_clean_names):
    if name in visited:
        continue
    cluster = [name]
    visited.add(name)
    for other in unique_clean_names:
        if other in visited:
            continue
        score = fuzz.token_sort_ratio(name, other)
        if score >= 90:
            cluster.append(other)
            visited.add(other)
    clusters.append(cluster)

grouped_data = []

for cluster in clusters:
    cluster_df = df[df['cleanName'].isin(cluster)]
    total_sales = cluster_df['totalProductPrice'].sum()
    total_qty = cluster_df['quantity'].sum()

    representative = cluster_df['productName'].mode().iloc[0]
    top_brand = cluster_df['brandName'].mode().iloc[0] if not cluster_df['brandName'].isna().all() else ''
    top_category = cluster_df['categoryName'].mode().iloc[0] if not cluster_df['categoryName'].isna().all() else ''

    grouped_data.append({
        'Product Name': representative,
        'Brand': top_brand,
        'Category': top_category,
        'Total Sales': round(total_sales, 2),
        'Quantity Sold': int(total_qty)
    })

result_df = pd.DataFrame(grouped_data)
result_df.sort_values(by='Total Sales', ascending=False, inplace=True)
result_df.insert(0, 'S.No', range(1, len(result_df) + 1))

MAX_ROWS = 1000000
num_sheets = math.ceil(len(result_df) / MAX_ROWS)

with pd.ExcelWriter("top_products_july2025.xlsx", engine='openpyxl') as writer:
    for i in range(num_sheets):
        sheet_df = result_df.iloc[i * MAX_ROWS : (i
                                                   + 1) * MAX_ROWS]
        sheet_df.to_excel(writer, sheet_name=f"Top Products {i+1}", index=False)

print("Exported to top_products_july2025.xlsx")

🔍 Clustering similar product names...


100%|██████████| 24524/24524 [18:36<00:00, 21.97it/s] 


Exported to top_products_july2025.xlsx


In [6]:
import pandas as pd
from sqlalchemy import create_engine
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import faiss
import numpy as np

In [ ]:
engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASS}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
)

query = """
SELECT productName, totalProductPrice, quantity, brandName, categoryName
FROM sales_data
WHERE orderDate BETWEEN '2025-02-01' and '2025-07-31'
AND productName IS NOT NULL
"""
df = pd.read_sql(query, engine)
df['totalProductPrice'] = pd.to_numeric(df['totalProductPrice'], errors='coerce')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df.dropna(subset=['totalProductPrice', 'quantity'], inplace=True)
df['text'] = df.apply(lambda row: f"{row['productName']} | {row['brandName']} | {row['categoryName']}", axis=1)

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print("Generating embeddings...")
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings)
index.add(embeddings)

threshold = 0.9
visited = set()
clusters = []

for i in range(len(embeddings)):
    if i in visited:
        continue
    _, neighbors = index.search(np.expand_dims(embeddings[i], axis=0), len(embeddings))
    cluster = []
    for j in neighbors[0]:
        if j not in visited and np.dot(embeddings[i], embeddings[j]) >= threshold:
            cluster.append(j)
            visited.add(j)
    clusters.append(cluster)

results = []

for cluster in clusters:
    cluster_df = df.iloc[cluster]
    total_sales = cluster_df['totalProductPrice'].sum()
    total_qty = cluster_df['quantity'].sum()
    top_name = cluster_df['productName'].mode().iloc[0]
    top_brand = cluster_df['brandName'].mode().iloc[0] if not cluster_df['brandName'].isna().all() else ''
    top_category = cluster_df['categoryName'].mode().iloc[0] if not cluster_df['categoryName'].isna().all() else ''

    results.append({
        "Product Name": top_name,
        "Brand": top_brand,
        "Category": top_category,
        "Total Sales": round(total_sales, 2),
        "Quantity Sold": int(total_qty)
    })

result_df = pd.DataFrame(results)
result_df.sort_values(by='Total Sales', ascending=False, inplace=True)
result_df.insert(0, 'S.No', range(1, len(result_df) + 1))

🔍 Generating embeddings...


Batches:   0%|          | 0/36653 [00:00<?, ?it/s]

c:\Users\it\Documents\sales_analysis_algorithm\vmac\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [8]:
result_df.to_excel("top_products_july2025_llm.xlsx", index=False)
print("Exported to top_products_july2025_llm.xlsx")

Exported to top_products_july2025_llm.xlsx


In [ ]:
if products:
    print(f"Database Summary:")
    print(f"   Total unique products: {len(products)}")
    print(f"   Sample products:")
    for i, product in enumerate(products[:10], 1):
        print(f"      {i}. {product}")
        
    if len(products) > 10:
        print(f"... and {len(products)-10} more")

Database Summary:
   Total unique products: 47490
   Sample products:
      1. 	MCVITTIES DIGSTV BAR CHOC 30GM
      2.           Slow Kiss Perfume for Men - Aromatic Fresh - 100ml
      3.      Zandu dant veer
      4.     lighter clipper
      5.    Marlboro Clove 
      6.    TEDDY MARSH MELLO 16G 
      7.   GHEE BATTI
      8.   SAATHI SPIRAL SOFT MRP180
      9.   Tofu Soya Paneer
      10.  1 pcs Classic Mild 
      ... and 47480 more


In [ ]:
class ProductSimilarityModel:
    """
    A model to find semantic similarity between products and group similar products together.
    This helps in analyzing product families rather than individual SKUs.
    """
    
    def __init__(self, similarity_threshold: float = 0.7):
        """Initialize the model with similarity threshold."""
        self.similarity_threshold = similarity_threshold
        self.vectorizer = None
        self.product_vectors = None
        self.product_names = None
        self.similarity_matrix = None
        self.product_groups = None
        self.group_representatives = None

    def preprocess_product_name(self, name: str) -> str:
        """Clean and preprocess product names for better similarity matching."""
        if pd.isna(name):
            return ""
        
        # Convert to lowercase
        name = name.lower()
        
        # Remove specific measurement patterns found in your data
        patterns_to_remove = [
            # Handle decimal measurements like 80.00gm, 1000.00ml, 21.00gm
            r'\b\d+\.\d+\s*(gm|ml|kg|mg|oz|lbs?|pcs?|pieces?|g)\b',
            
            # Handle integer measurements like 250ml, 20pcs, 10s
            r'\b\d+\s*(gm|ml|kg|mg|oz|lbs?|pcs?|pieces?|g)\b',
            
            # Handle measurements without decimal like 250ml, 20pcs
            r'\b\d+\s*(ml|mg|g|gm|kg|oz|lbs?|pcs?|pieces?|pack|box|can|bottle|packet)\b',
            
            # Handle combinations like "250ml x 6", "10 x 20gm"
            r'\b\d+(\.\d+)?\s*x\s*\d+(\.\d+)?\s*(ml|mg|g|gm|kg|oz|lbs?|pcs?)\b',
            
            # Handle size indicators
            r'\b(small|medium|large|xl|xs|s|m|l)\b',
            
            # Handle standalone numbers like "10s", "20", "250"
            r'\b\d+s?\b',
            
            # Handle specific patterns like "10S", "20pcs"
            r'\b\d+[sS]\b',
            
            # Handle weight/volume without units that might be left
            r'\b\d+(\.\d+)?\b',
        ]
        
        for pattern in patterns_to_remove:
            name = re.sub(pattern, '', name)
        
        # Normalize common terms
        name = re.sub(r'\bcigarette\b', 'cigarette', name)
        name = re.sub(r'\bcigarettes\b', 'cigarette', name)
        name = re.sub(r'\benergy drink\b', 'energy drink', name)
        
        # Remove extra spaces and special characters except spaces
        name = re.sub(r'[^\w\s]', ' ', name)
        name = re.sub(r'\s+', ' ', name).strip()
        
        return name

    def extract_features(self, products: List[str]) -> np.ndarray:
        """Extract TF-IDF features from product names."""
        # Preprocess product names
        processed_products = [self.preprocess_product_name(name) for name in products]
        
        # Initialize TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 3),
            stop_words='english',
            min_df=1,
            max_df=0.8,
            lowercase=True,
            token_pattern=r'\b[a-zA-Z][a-zA-Z]+\b'
        )
        
        # Fit and transform the data
        self.product_vectors = self.vectorizer.fit_transform(processed_products)
        
        return self.product_vectors
    
    def calculate_similarity_matrix(self) -> np.ndarray:
        """Calculate cosine similarity matrix between all products."""
        if self.product_vectors is None:
            raise ValueError("Please extract features first using extract_features()")
            
        self.similarity_matrix = cosine_similarity(self.product_vectors)
        return self.similarity_matrix
    
    def find_similar_products(self, product_index: int, top_k: int = 10) -> List[Tuple[int, str, float]]:
        """Find top-k similar products for a given product."""
        if self.similarity_matrix is None:
            self.calculate_similarity_matrix()
            
        similarities = self.similarity_matrix[product_index]
        
        similar_indices = np.argsort(similarities)[::-1][1:top_k+1]
        
        results = []
        for idx in similar_indices:
            if similarities[idx] >= self.similarity_threshold:
                results.append((idx, self.product_names[idx], similarities[idx]))
                
        return results
    
    def group_similar_products(self) -> Dict[int, List[int]]:
        """Group similar products using clustering."""
        if self.similarity_matrix is None:
            self.calculate_similarity_matrix()
        
        similarity_matrix = np.clip(self.similarity_matrix, 0, 1)
        distance_matrix = 1 - similarity_matrix

        eps = 1 - self.similarity_threshold
        clustering = DBSCAN(eps=eps, min_samples=1, metric='precomputed')
        cluster_labels = clustering.fit_predict(distance_matrix)

        self.product_groups = {}
        for idx, label in enumerate(cluster_labels):
            if label not in self.product_groups:
                self.product_groups[label] = []
            self.product_groups[label].append(idx)

        return self.product_groups
    
    def get_group_representatives(self) -> Dict[int, Tuple[int, str]]:
        """Get representative product for each group."""
        if self.product_groups is None:
            self.group_similar_products()
            
        self.group_representatives = {}
        
        for group_id, product_indices in self.product_groups.items():
            if len(product_indices) == 1:
                # Single product group
                idx = product_indices[0]
                self.group_representatives[group_id] = (idx, self.product_names[idx])
            else:
                # Find product with highest average similarity to other products in group
                best_idx = None
                best_avg_similarity = -1
                
                for idx in product_indices:
                    avg_similarity = np.mean([self.similarity_matrix[idx][other_idx] 
                                            for other_idx in product_indices if other_idx != idx])
                    if avg_similarity > best_avg_similarity:
                        best_avg_similarity = avg_similarity
                        best_idx = idx
                
                self.group_representatives[group_id] = (best_idx, self.product_names[best_idx])
        
        return self.group_representatives
    
    def fit(self, products: List[str]) -> 'ProductSimilarityModel':
        """Fit the model on product data."""
        self.product_names = products
        
        print(f"Processing {len(products)} products...")
        
        # Extract features
        self.extract_features(products)
        print("Features extracted")
        
        # Calculate similarity matrix
        self.calculate_similarity_matrix()
        print("Similarity matrix calculated")
        
        # Group similar products
        self.group_similar_products()
        print(f"Products grouped into {len(self.product_groups)} groups")
        
        # Get group representatives
        self.get_group_representatives()
        print("Group representatives identified")
        
        return self
    
    def get_consolidated_products(self, sales_data: pd.DataFrame, 
                                product_col: str = 'Product Name',
                                sales_col: str = 'Sales',
                                quantity_col: str = 'Quantity Sold') -> pd.DataFrame:
        """Consolidate similar products and aggregate their sales/quantities."""
        if self.product_groups is None or self.group_representatives is None:
            raise ValueError("Please fit the model first")
        
        product_to_representative = {}
        
        for group_id, product_indices in self.product_groups.items():
            representative_idx, representative_name = self.group_representatives[group_id]
            
            for product_idx in product_indices:
                original_product = self.product_names[product_idx]
                product_to_representative[original_product] = representative_name
        
        consolidated_data = sales_data.copy()
        consolidated_data['Consolidated_Product'] = consolidated_data[product_col].map(
            product_to_representative
        ).fillna(consolidated_data[product_col])
        
        aggregated_data = consolidated_data.groupby('Consolidated_Product').agg({
            sales_col: 'sum',
            quantity_col: 'sum'
        }).reset_index()

        aggregated_data.columns = ['Product Name', 'Sales', 'Quantity Sold']
        aggregated_data = aggregated_data.sort_values('Sales', ascending=False).reset_index(drop=True)
        aggregated_data.insert(0, 'S.No', range(1, len(aggregated_data) + 1))

        return aggregated_data

    def analyze_groups(self) -> pd.DataFrame:
        """Analyze the groups formed and return summary statistics."""
        if self.product_groups is None:
            raise ValueError("Please fit the model first")

        analysis_data = []

        for group_id, product_indices in self.product_groups.items():
            representative_idx, representative_name = self.group_representatives[group_id]

            group_products = [self.product_names[idx] for idx in product_indices]

            analysis_data.append({
                'Group_ID': group_id,
                'Representative_Product': representative_name,
                'Group_Size': len(product_indices),
                'Products_in_Group': ' | '.join(group_products)
            })

        analysis_df = pd.DataFrame(analysis_data)
        analysis_df = analysis_df.sort_values('Group_Size', ascending=False).reset_index(drop=True)

        return analysis_df

In [ ]:
def test_preprocessing():
    """Test preprocessing with sample products."""
    print("TESTING PREPROCESSING WITH YOUR PRODUCT PATTERNS")
    print("="*70)

    # Use actual samples from your data if available
    if products:
        test_products = products[:20]
    else:
        # Fallback sample products based on your patterns
        test_products = [
            "Red Bull Energy Drink 250ml Can",
            "Red Bull Energy Drink 80.00ml",
            "Marlboro Advance Compact 10S", 
            "Marlboro Advance Compact 21.00gm",
            "Wills Classic Connect Cigarette 20pcs",
            "Wills Classic Connect Cigarette 50.00gm",
            "Gold Flake Premium 10Pcs",
            "Gold Flake Premium 1.00pcs"
        ]
    
    model = ProductSimilarityModel()
    
    preprocessing_results = []
    print("Original Product Name → Preprocessed Name")
    print("-" * 70)
    
    for product in test_products:
        preprocessed = model.preprocess_product_name(product)
        preprocessing_results.append({
            'Original': product,
            'Preprocessed': preprocessed
        })
        print(f"{product:<35} → {preprocessed}")
    
    return pd.DataFrame(preprocessing_results)

In [39]:
# Run preprocessing test
preprocessing_df = test_preprocessing()

🧪 TESTING PREPROCESSING WITH YOUR PRODUCT PATTERNS

Original Product Name → Preprocessed Name
----------------------------------------------------------------------
	MCVITTIES DIGSTV BAR CHOC 30GM     → mcvitties digstv bar choc
          Slow Kiss Perfume for Men - Aromatic Fresh - 100ml → slow kiss perfume for men aromatic fresh
     Zandu dant veer                → zandu dant veer
    lighter clipper                 → lighter clipper
   Marlboro Clove                   → marlboro clove
   TEDDY MARSH MELLO 16G            → teddy marsh mello
  GHEE BATTI                        → ghee batti
  SAATHI SPIRAL SOFT MRP180         → saathi spiral soft mrp180
  Tofu Soya Paneer                  → tofu soya paneer
 1 pcs Classic Mild                 → classic mild
 45 San Vito Pasta Plain            → san vito pasta plain
 4700 Microwave Popcorn Natural Original Healthy Popcorn 85G → microwave popcorn natural original healthy popcorn
 8 -IN -1 MIX SEED&NUTS  250 GM     → in mix seed nuts
 AC

In [ ]:
def train_and_compare_thresholds(products_to_use, thresholds=[0.5, 0.6, 0.7, 0.8]):
    """Train models with different similarity thresholds and compare results."""
    print(f"COMPARING SIMILARITY THRESHOLDS")
    print("="*70)

    results = []
    models = {}

    for threshold in thresholds:
        print(f"Training model with threshold {threshold}...")
        
        model = ProductSimilarityModel(similarity_threshold=threshold)
        model.fit(products_to_use)

        analysis_df = model.analyze_groups()
        multi_groups = len(analysis_df[analysis_df['Group_Size'] > 1])
        single_groups = len(analysis_df[analysis_df['Group_Size'] == 1])
        reduction_pct = ((len(products_to_use) - len(analysis_df)) / len(products_to_use)) * 100

        results.append({
            'Threshold': threshold,
            'Total_Groups': len(analysis_df),
            'Multi_Product_Groups': multi_groups,
            'Single_Product_Groups': single_groups,
            'Reduction_Percentage': reduction_pct
        })

        models[threshold] = model
        print(f"Created {len(analysis_df)} groups ({multi_groups} multi-product)")

    comparison_df = pd.DataFrame(results)
    print(f"THRESHOLD COMPARISON RESULTS:")
    print(comparison_df.to_string(index=False))
    
    return comparison_df, models

In [ ]:
products_subset = products[:100] if len(products) > 100 else products
threshold_comparison, trained_models = train_and_compare_thresholds(products_subset)


🎚️ COMPARING SIMILARITY THRESHOLDS

🔄 Training model with threshold 0.5...
🔄 Processing 100 products...
✅ Features extracted
✅ Similarity matrix calculated
✅ Products grouped into 98 groups
✅ Group representatives identified
   ✅ Created 98 groups (2 multi-product)

🔄 Training model with threshold 0.6...
🔄 Processing 100 products...
✅ Features extracted
✅ Similarity matrix calculated
✅ Products grouped into 99 groups
✅ Group representatives identified
   ✅ Created 99 groups (1 multi-product)

🔄 Training model with threshold 0.7...
🔄 Processing 100 products...
✅ Features extracted
✅ Similarity matrix calculated
✅ Products grouped into 99 groups
✅ Group representatives identified
   ✅ Created 99 groups (1 multi-product)

🔄 Training model with threshold 0.8...
🔄 Processing 100 products...
✅ Features extracted
✅ Similarity matrix calculated
✅ Products grouped into 99 groups
✅ Group representatives identified
   ✅ Created 99 groups (1 multi-product)

📊 THRESHOLD COMPARISON RESULTS:
 Thresh

In [56]:
best_threshold = 0.6

print(f"USING MODEL WITH THRESHOLD {best_threshold}")
print("="*70)

USING MODEL WITH THRESHOLD 0.6


In [ ]:
best_model = trained_models[best_threshold]

analysis_df = best_model.analyze_groups()

In [ ]:
print(f"GROUP ANALYSIS SUMMARY:")
print(f"Total products analyzed: {len(products_subset)}")
print(f"Total groups created: {len(analysis_df)}")
print(f"Multi-product groups: {len(analysis_df[analysis_df['Group_Size'] > 1])}")
print(f"Products consolidated: {len(products_subset) - len(analysis_df)}")
print(f"Reduction percentage: {((len(products_subset) - len(analysis_df)) / len(products_subset) * 100):.1f}%")

print(f"TOP MULTI-PRODUCT GROUPS:")
multi_groups = analysis_df[analysis_df['Group_Size'] > 1].head(10)

GROUP ANALYSIS SUMMARY:
   Total products analyzed: 100
   Total groups created: 99
   Multi-product groups: 1
   Products consolidated: 1
   Reduction percentage: 1.0%

🔗 TOP MULTI-PRODUCT GROUPS:


In [ ]:
for idx, row in multi_groups.iterrows():
    print(f"Group {row['Group_ID']} ({row['Group_Size']} products)")
    print(f"   Representative: {row['Representative_Product']}")
    products_in_group = row['Products_in_Group'].split(' | ')
    for i, product in enumerate(products_in_group, 1):
        print(f"{i}. {product}")

Group -1 (2 products)
   Representative:  ewen500g
      1.  ewen500g
      2.  GRENDER1600


In [ ]:
if not sample_sales.empty:
    print(f"TESTING CONSOLIDATION WITH REAL SALES DATA")
    print("="*70)

    print(f"Original Sales Data (Top 15):")
    display_cols = ['S.No', 'Product Name', 'Sales', 'Quantity Sold']
    print(sample_sales[display_cols].head(15).to_string(index=False))

    try:
        consolidated_sales = best_model.get_consolidated_products(
            sample_sales,
            product_col='Product Name',
            sales_col='Sales', 
            quantity_col='Quantity Sold'
        )

        print(f"Consolidated Sales Data (Top 15):")
        print(consolidated_sales[display_cols].head(15).to_string(index=False))
        print(f"CONSOLIDATION IMPACT:")
        print(f"Original products: {len(sample_sales)}")
        print(f"Consolidated products: {len(consolidated_sales)}")
        print(f"Reduction: {((len(sample_sales) - len(consolidated_sales)) / len(sample_sales) * 100):.1f}%")

        original_total = sample_sales['Sales'].sum()
        consolidated_total = consolidated_sales['Sales'].sum()
        print(f"Original total sales: ₹{original_total:,.2f}")
        print(f"Consolidated total sales: ₹{consolidated_total:,.2f}")
        print(f"Sales preservation: {(consolidated_total/original_total*100):.1f}%")

    except Exception as e:
        print(f"Consolidation test failed: {e}")
else:
    print("No sample sales data available for consolidation testing")

TESTING CONSOLIDATION WITH REAL SALES DATA

📈 Original Sales Data (Top 15):


KeyError: "['Sales'] not in index"

In [ ]:
def standardize_sales_data(df):
    """
    Standardize column names to work with the consolidation logic.
    Ensures presence of: 'S.No', 'Product Name', 'Sales', 'Quantity Sold'
    """
    rename_map = {}

    for col in df.columns:
        col_lower = col.strip().lower()
        if 'product' in col_lower:
            rename_map[col] = 'Product Name'
        elif 'price' in col_lower or 'sales' in col_lower:
            rename_map[col] = 'Sales'
        elif 'qty' in col_lower or 'quantity' in col_lower:
            rename_map[col] = 'Quantity Sold'
    
    df = df.rename(columns=rename_map)
    
    if 'S.No' not in df.columns:
        df.insert(0, 'S.No', range(1, len(df) + 1))
    
    return df

def test_product_consolidation(sample_sales, best_model):
    """
    Run and print the product consolidation test using the provided model and sales data.
    """
    if sample_sales.empty:
        print("No sample sales data available for consolidation testing")
        return

    sample_sales = standardize_sales_data(sample_sales)

    print(f"TESTING CONSOLIDATION WITH REAL SALES DATA")
    print("=" * 70)

    try:
        display_cols = ['S.No', 'Product Name', 'Sales', 'Quantity Sold']
        print(f"Original Sales Data (Top 15):")
        print(sample_sales[display_cols].head(15).to_string(index=False))

        consolidated_sales = best_model.get_consolidated_products(
            sample_sales,
            product_col='Product Name',
            sales_col='Sales', 
            quantity_col='Quantity Sold'
        )

        print(f"Consolidated Sales Data (Top 15):")
        print(consolidated_sales[display_cols].head(15).to_string(index=False))

        print(f"CONSOLIDATION IMPACT:")
        print(f"Original products: {len(sample_sales)}")
        print(f"Consolidated products: {len(consolidated_sales)}")
        reduction_pct = ((len(sample_sales) - len(consolidated_sales)) / len(sample_sales)) * 100
        print(f"Reduction: {reduction_pct:.1f}%")

        original_total = sample_sales['Sales'].sum()
        consolidated_total = consolidated_sales['Sales'].sum()
        print(f"Original total sales: ₹{original_total:,.2f}")
        print(f"Consolidated total sales: ₹{consolidated_total:,.2f}")
        print(f"Sales preservation: {(consolidated_total / original_total * 100):.1f}%")

    except Exception as e:
        print(f"Consolidation test failed: {e}")
        print("Columns in sample_sales:", sample_sales.columns.tolist())

In [ ]:
def create_visualizations(threshold_comparison, analysis_df, sample_sales, consolidated_sales):
    """Create visualizations for the model results."""

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Groups vs Threshold', 'Reduction % vs Threshold', 
                       'Group Size Distribution', 'Sales Comparison'),
        specs=[[{"secondary_y": True}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    fig.add_trace(
        go.Scatter(x=threshold_comparison['Threshold'], 
                  y=threshold_comparison['Total_Groups'],
                  name='Total Groups', line=dict(color='blue')),
                row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=threshold_comparison['Threshold'], 
                  y=threshold_comparison['Multi_Product_Groups'],
                  name='Multi-Product Groups', line=dict(color='red')),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(x=threshold_comparison['Threshold'], 
               y=threshold_comparison['Reduction_Percentage'],
               name='Reduction %', marker_color='green'),
        row=1, col=2
    )

    group_sizes = analysis_df['Group_Size'].value_counts().sort_index()
    fig.add_trace(
        go.Bar(x=group_sizes.index, y=group_sizes.values,
               name='Group Frequency', marker_color='orange'),
        row=2, col=1
    )

    if not sample_sales.empty and 'consolidated_sales' in locals():
        fig.add_trace(
            go.Bar(x=['Original', 'Consolidated'], 
                   y=[len(sample_sales), len(consolidated_sales)],
                   name='Product Count', marker_color='purple'),
            row=2, col=2
    )

    fig.update_layout(height=800, showlegend=True, 
                     title_text="Product Similarity Model Analysis Results")
    fig.show()

In [61]:
test_product_consolidation(sample_sales, best_model)


📈 TESTING CONSOLIDATION WITH REAL SALES DATA

📈 Original Sales Data (Top 15):
❌ Consolidation test failed: "['Sales'] not in index"
🧾 Columns in sample_sales: ['S.No', 'Product Name', 'Product Name', 'Quantity Sold']


In [ ]:
# visualize
try:
    if 'consolidated_sales' in locals():
        create_visualizations(threshold_comparison, analysis_df, sample_sales, consolidated_sales)
    else:
        print("Visualization skipped - consolidated sales data not available")
except Exception as e:
    print(f"Visualization error: {e}")

📊 Visualization skipped - consolidated sales data not available


In [ ]:
## save model function
def save_model(model, threshold):
    """Save the trained model to disk."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'product_similarity_model_t{threshold}_{timestamp}.pkl'

    try:
        model_data = {
            'vectorizer': model.vectorizer,
            'product_names': model.product_names,
            'product_groups': model.product_groups,
            'group_representatives': model.group_representatives,
            'similarity_threshold': model.similarity_threshold,
            'training_date': datetime.now(),
            'total_products': len(model.product_names)
        }

        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)

        print(f"Model saved successfully as: {filename}")
        return filename

    except Exception as e:
        print(f"Error saving model: {e}")
        return None

In [ ]:
model_filename = save_model(best_model, best_threshold)
print(f"MODEL TRAINING COMPLETED!")
print("="*70)

NameError: name 'best_model' is not defined

In [ ]:
### Notes


1. Load the model in your analysis.py:
   ```python
   from product_similarity_model import ProductSimilarityModel
   model = ProductSimilarityModel.load_model('{model_filename}')
   ```

2. Replace your product data fetching:
   ```python
   # Instead of:
   product_data = fetch_product_data(start_date, end_date)
   
   # Use:
   consolidated_data = model.get_consolidated_products(
       your_sales_data, 
       product_col='Product Name',
       sales_col='Sales',
       quantity_col='Quantity Sold'
   )
   ```

3. Your dashboard will now show consolidated product families instead of individual SKUs!
""")

print(f"\n🎯 MODEL PERFORMANCE SUMMARY:")
if 'consolidated_sales' in locals():
    print(f"   • Reduced {len(sample_sales)} products to {len(consolidated_sales)} ({((len(sample_sales) - len(consolidated_sales)) / len(sample_sales) * 100):.1f}% reduction)")
    print(f"   • Sales data fully preserved (100% accuracy)")
print(f"   • Similarity threshold: {best_threshold}")
print(f"   • Total groups created: {len(analysis_df)}")
print(f"   • Multi-product groups: {len(analysis_df[analysis_df['Group_Size'] > 1])}")

print(f"\n✨ Your product analysis will now focus on product families rather than individual package sizes!")